In [18]:
"""
FASE 1: Problema Directo con Efectos de Surfactante
====================================================
Modelo simplificado 2D basado en Avilés-Rojas & Hurtado (2025)

Ecuaciones:
- Equilibrio: Div(F·S) = 0
- Tensor de esfuerzos: S = S_el + (P_γ - P_alv)·J·C⁻¹
- Presión de colapso: P_γ = (2γ/R_I)·(Φ_R/Φ)^(1/3)

Simplificaciones:
- 2D (estado plano de deformación)
- Quasi-estático (sin dinámica de Darcy)
- Surfactante en régimen insoluble (γ función de Γ)
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
import numpy as np
from firedrake.output import VTKFile
import matplotlib.pyplot as plt

fd.parameters["form_compiler"]["quadrature_degree"] = 6

# ============================================================
# 1. PARÁMETROS DEL MODELO (Tabla 1 del paper)
# ============================================================

# Geometría alveolar
Phi_R = 0.74          # Porosidad de referencia [-]
R_RI = 0.11           # Radio interno de referencia [mm]

# Material Fung (tejido)
c_fung = 2.5       # [kPa] (paper usa 2.5 kPa, escalamos para 2D)
a_fung = 0.433        # [-]
b_fung = -0.61        # [-]

# Surfactante (Otis et al.)
gamma_0 = 70.0e-6     # Tensión superficial sin surfactante [N/mm = kPa·mm]
gamma_inf = 22.2e-6   # γ en umbral Langmuir/insoluble [N/mm]
gamma_min = 0.0       # Tensión mínima [N/mm]
Gamma_inf = 3.0e-9    # Concentración umbral [g/mm²]
m1 = 47.8e-6          # Pendiente régimen Langmuir [N/mm]
m2 = 140.0e-6         # Pendiente régimen insoluble [N/mm]

# Concentración máxima dinámica (Ec. 51)
Gamma_max = Gamma_inf * (1.0 + (gamma_inf - gamma_min) / m2)

print("="*60)
print("PARÁMETROS DEL MODELO")
print("="*60)
print(f"Φ_R = {Phi_R}, R_I = {R_RI} mm")
print(f"Fung: c={c_fung} kPa, a={a_fung}, b={b_fung}")
print(f"Surfactante: γ₀={gamma_0*1e6:.1f}, γ_inf={gamma_inf*1e6:.1f} μN/mm")
print(f"Γ_inf={Gamma_inf:.2e}, Γ_max={Gamma_max:.2e} g/mm²")

# ============================================================
# 2. MALLA Y ESPACIOS DE FUNCIONES
# ============================================================

# Dominio: lámina rectangular (representa parénquima)
Lx, Ly = 10.0, 10.0  # [mm]
nx, ny = 20, 20

mesh = fd.RectangleMesh(nx, ny, Lx, Ly)
print(f"\nMalla: {nx}x{ny} elementos, dominio {Lx}x{Ly} mm")

V = fd.VectorFunctionSpace(mesh, "CG", 2) # Desplazamiento
Q = fd.FunctionSpace(mesh, "DG", 0)       # Concentración surfactante (por elemento)

u = fd.Function(V, name="Displacement")
v = fd.TestFunction(V)
Gamma = fd.Function(Q, name="Surfactant_Concentration")

coords_initial = mesh.coordinates.dat.data_ro.copy()

dim = mesh.geometric_dimension()
I = fd.Identity(dim)

# ============================================================
# 3. MODELO CONSTITUTIVO
# ============================================================

# Cinemática
F = I + fd.grad(u)
J = fd.det(F)
C = F.T * F
invC = fd.inv(C)
# Tensor de Green-Lagrange
E = fd.variable(0.5 * (C - I))

# Invariantes de E
J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))

# Energía elástica Fung (Ec. 44)
Psi_el = c_fung * (fd.exp(a_fung * J1**2 + b_fung * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

# Porosidad Lagrangiana: Φ = J - 1 + Φ_R (Ec. 17)
Phi = fd.max_value(J - 1.0 + Phi_R, 1e-4)

# Tensión superficial según modelo de Otis (Ec. 48)
ratio = Gamma / Gamma_inf
gamma_lang = gamma_0 - m1 * ratio
gamma_insol = gamma_inf - m2 * (ratio - 1.0)
gamma = fd.conditional(fd.lt(Gamma, Gamma_inf), gamma_lang, gamma_insol)
gamma = fd.max_value(gamma, gamma_min)

# Presión de colapso: P_γ = (2γ/R_I)·(Φ_R/Φ)^(1/3) (Ec. 43)
P_gamma = (2.0 * gamma / R_RI) * ((Phi_R / Phi)**(1.0/3.0))

P_alv = fd.Constant(0.0)

# Tensor de esfuerzos total (Ec. 42) S = S_el + (P_γ - P_alv)·J·C⁻¹
S_total = S_el + (P_gamma - P_alv) * J * invC

# ============================================================
# 4. FORMULACIÓN VARIACIONAL
# ============================================================

# Residuo mecánico (Ec. 70)
F_stress = F * S_total
Residual = fd.inner(F_stress, fd.grad(v)) * fd.dx

bcs = [
    fd.DirichletBC(V, fd.Constant((0.0, 0.0)), 1),
    fd.DirichletBC(V.sub(1), 0.0, 3),
]

problem = fd.NonlinearVariationalProblem(Residual, u, bcs=bcs)

solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt",
    "snes_max_it": 100,
    "snes_atol": 1e-8,
    "snes_rtol": 1e-6,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}

solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# ============================================================
# 5. SIMULACIÓN DE INFLACIÓN
# ============================================================

print("\n" + "="*60)
print("SIMULACIÓN DE INFLACIÓN")
print("="*60)

Gamma.assign(fd.Constant(Gamma_max))
u.assign(fd.Constant((0.0, 0.0)))

output_dir = "phase1_results"
os.makedirs(output_dir, exist_ok=True)

gamma_field = fd.Function(Q, name="Surface_Tension")
P_gamma_field = fd.Function(Q, name="Collapse_Pressure")
J_field = fd.Function(Q, name="Jacobian")

P_target = 1.0 # 1.0 kPa de carga máxima
n_steps = 40

steps_list = [0]
pressures = [0.0]
volumes = [0.0]
u_norms = [0.0]
energies = [0.0]

outfile = VTKFile(os.path.join(output_dir, "inflation.pvd"))

for step in range(1, n_steps + 1):
    P_val = P_target * step / n_steps
    P_alv.assign(P_val)
    
    print(f"\nStep {step}/{n_steps}: P_alv = {P_val:.4f} kPa")
    
    try:
        solver.solve()
    except fd.ConvergenceError:
        print("  ⚠ No convergió")
        break
    
    # Actualizar surfactante
    J_curr = fd.project(fd.det(I + fd.grad(u)), Q)
    Phi_curr = fd.project(fd.max_value(J_curr - 1.0 + Phi_R, 1e-4), Q)
    
    # Densidad de área interfacial: A = (3/R_I)·Φ_R^(1/3)·Φ^(2/3) (Ec. 39)
    # Ratio = (Phi_R / Phi)^(2/3)
    area_ratio = (Phi_R / Phi_curr)**(2.0/3.0)
    Gamma_new = fd.project(Gamma_max * area_ratio, Q)
    Gamma_new_data = np.clip(Gamma_new.dat.data[:], Gamma_inf, Gamma_max)
    Gamma.dat.data[:] = Gamma_new_data
    
    # Campos
    J_field.assign(J_curr)
    
    ratio_curr = Gamma / Gamma_inf
    gamma_expr = fd.conditional(fd.lt(Gamma, Gamma_inf), 
                                 gamma_0 - m1 * ratio_curr,
                                 gamma_inf - m2 * (ratio_curr - 1.0))
    gamma_expr = fd.max_value(gamma_expr, gamma_min)
    gamma_field.assign(fd.project(gamma_expr, Q))
    
    Phi_safe = fd.max_value(J_curr - 1.0 + Phi_R, 1e-4)
    P_gamma_expr = (2.0 * gamma_expr / R_RI) * ((Phi_R / Phi_safe)**(1.0/3.0))
    P_gamma_field.assign(fd.project(P_gamma_expr, Q))
    
    # Métricas
    V_change = fd.assemble((J_curr - 1.0) * fd.dx)
    u_L2 = fd.norm(u)
    energy = fd.assemble(Psi_el * fd.dx)
    
    steps_list.append(step)
    pressures.append(P_val)
    volumes.append(V_change)
    u_norms.append(u_L2)
    energies.append(energy)
    
    J_min, J_max = J_field.dat.data_ro.min(), J_field.dat.data_ro.max()
    gamma_avg = fd.assemble(gamma_field * fd.dx) / (Lx * Ly)
    
    print(f"  ||u||_L2 = {u_L2:.6e}, J ∈ [{J_min:.4f}, {J_max:.4f}]")
    print(f"  γ_avg = {gamma_avg*1e6:.2f} μN/mm, ΔV = {V_change:.4f} mm²")
    
    outfile.write(u, Gamma, gamma_field, P_gamma_field, J_field)

# ============================================================
# 6. GUARDAR MALLA DEFORMADA
# ============================================================

print("\n" + "="*60)
print("GUARDANDO RESULTADOS")
print("="*60)

V_coords = fd.VectorFunctionSpace(mesh, "CG", 1)
# Aquí saltará el warning de interpolate, es normal.
u_at_nodes = fd.interpolate(u, V_coords)
coords_deformed = coords_initial + u_at_nodes.dat.data_ro

mesh_deformed = fd.Mesh(mesh.coordinates.copy(deepcopy=True))
mesh_deformed.coordinates.dat.data[:] = coords_deformed

deformed_file = VTKFile(os.path.join(output_dir, "mesh_deformed.pvd"))
V_def = fd.VectorFunctionSpace(mesh_deformed, "CG", 1)
zero_field = fd.Function(V_def, name="zero")
deformed_file.write(zero_field)

np.savez(os.path.join(output_dir, "forward_data.npz"),
         u_data=u.dat.data_ro[:],
         Gamma_data=Gamma.dat.data_ro[:],
         P_alv_final=float(P_alv),
         coords_initial=coords_initial,
         coords_deformed=coords_deformed,
         steps=np.array(steps_list),
         pressures=np.array(pressures),
         volumes=np.array(volumes),
         u_norms=np.array(u_norms),
         energies=np.array(energies))

# ============================================================
# 7. GRÁFICAS
# ============================================================

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].plot(pressures, volumes, 'b-o', markersize=4)
axes[0, 0].set_xlabel('Presión [kPa]')
axes[0, 0].set_ylabel('ΔV [mm²]')
axes[0, 0].set_title('Curva P-V')
axes[0, 0].grid(True)

axes[0, 1].plot(steps_list, u_norms, 'r-o', markersize=4)
axes[0, 1].set_xlabel('Paso')
axes[0, 1].set_ylabel('||u||_L2 [mm]')
axes[0, 1].set_title('Norma L2 del Desplazamiento')
axes[0, 1].grid(True)

axes[1, 0].plot(steps_list[1:], energies[1:], 'g-o', markersize=4)
axes[1, 0].set_xlabel('Paso')
axes[1, 0].set_ylabel('Energía [kPa·mm²]')
axes[1, 0].set_title('Energía Elástica')
axes[1, 0].grid(True)

axes[1, 1].plot(coords_initial[:, 0], coords_initial[:, 1], 'b.', markersize=1, alpha=0.5, label='Inicial')
axes[1, 1].plot(coords_deformed[:, 0], coords_deformed[:, 1], 'r.', markersize=1, alpha=0.5, label='Deformado')
axes[1, 1].set_xlabel('x [mm]')
axes[1, 1].set_ylabel('y [mm]')
axes[1, 1].set_title('Configuración Inicial vs Deformada')
axes[1, 1].legend()
axes[1, 1].set_aspect('equal')
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, "convergence_plots.png"), dpi=150)
plt.close()

# ============================================================
# 8. RESUMEN
# ============================================================

print(f"\nArchivos en: {output_dir}/")

displacement_magnitude = np.linalg.norm(coords_deformed - coords_initial, axis=1)

print("\n" + "="*60)
print("RESUMEN")
print("="*60)
print(f"  Pasos: {len(steps_list)-1}/{n_steps}")
print(f"  P_alv final: {pressures[-1]:.4f} kPa")
print(f"  ||u||_L2 final: {u_norms[-1]:.6e} mm")
print(f"  ΔV final: {volumes[-1]:.4f} mm²")
print(f"  Desplazamiento máx: {displacement_magnitude.max():.6e} mm")
print(f"  Desplazamiento prom: {displacement_magnitude.mean():.6e} mm")

print("\nFASE 1 COMPLETADA")

PARÁMETROS DEL MODELO
Φ_R = 0.74, R_I = 0.11 mm
Fung: c=2.5 kPa, a=0.433, b=-0.61
Surfactante: γ₀=70.0, γ_inf=22.2 μN/mm
Γ_inf=3.00e-09, Γ_max=3.48e-09 g/mm²

Malla: 20x20 elementos, dominio 10.0x10.0 mm


KeyboardInterrupt: 

In [ ]:
"""
FASE 2 (DEFINITIVA): Recuperación visualizable en ParaView
==========================================================
Estrategia: Inverse Motion sobre Geometría Deformada

Objetivo:
  Generar un archivo .pvd que muestre la GEOMETRÍA DEFORMADA
  y contenga un vector 'Recovery_Vector' para recuperar la forma original.

Uso en ParaView:
  1. Abrir 'inverse_visual_ready.pvd'.
  2. Verás la malla distorsionada (deformada).
  3. Aplica filtro 'Warp by Vector' seleccionando 'Recovery_Vector'.
  4. La malla volverá a su forma cuadrada original.
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import firedrake as fd
import numpy as np
from firedrake.output import VTKFile

# ============================================================
# 1. PARÁMETROS FÍSICOS
# ============================================================

# Geometría y Material
Phi_R = 0.74
R_RI = 0.11
c_fung = 2.5
a_fung = 0.433
b_fung = -0.61

# Surfactante
gamma_0 = 70.0e-6
gamma_inf = 22.2e-6
gamma_min = 0.0
Gamma_inf = 3.0e-9
m1 = 47.8e-6
m2 = 140.0e-6

print("="*60)
print("FASE 2: INVERSE MOTION SOBRE MALLA DEFORMADA")
print("="*60)

# ============================================================
# 2. CARGAR "IMAGEN" (DATOS DE LA MALLA DEFORMADA)
# ============================================================

input_dir = "phase1_results"
if not os.path.exists(input_dir):
    raise FileNotFoundError(f"No se encuentra {input_dir}. Ejecuta phase1_forward.py primero.")

# Cargamos los datos numéricos de la Fase 1
print(f"Cargando datos desde {input_dir}/forward_data.npz ...")
data = np.load(os.path.join(input_dir, "forward_data.npz"))

# 1. Coordenadas de la malla deformada
coords_deformed = data['coords_deformed']

# 2. Estado físico asociado
Gamma_obs_data = data['Gamma_data']  
P_alv_final = float(data['P_alv_final'])

print(f"Entradas cargadas:")
print(f"  - Geometría: {coords_deformed.shape[0]} nodos")
print(f"  - Carga externa: P_alv = {P_alv_final*1000:.1f} Pa")

# ============================================================
# 3. CONSTRUCCIÓN DE LA MALLA OBSERVADA
# ============================================================

Lx, Ly = 10.0, 10.0
nx, ny = 20, 20

# 1. Topología base
mesh_topology = fd.RectangleMesh(nx, ny, Lx, Ly)

# 2. Malla deformada (Input del problema)
mesh_obs = fd.Mesh(mesh_topology.coordinates)
mesh_obs.coordinates.dat.data[:] = coords_deformed

print("Malla deformada reconstruida exitosamente.")

# ============================================================
# 4. ESPACIOS Y VARIABLES
# ============================================================

V = fd.VectorFunctionSpace(mesh_obs, "CG", 2)
Q = fd.FunctionSpace(mesh_obs, "DG", 0)

# Incógnita: w (Desplazamiento Inverso)
w = fd.Function(V, name="Inverse_Displacement")
v_test = fd.TestFunction(V)

# Campos fijos (Observados)
Gamma_field = fd.Function(Q, name="Gamma_Observed")
Gamma_field.dat.data[:] = Gamma_obs_data
alpha_load = fd.Constant(0.0) 

# ============================================================
# 5. FORMULACIÓN (CINEMÁTICA Y EQUILIBRIO)
# ============================================================

dim = mesh_obs.geometric_dimension()
I = fd.Identity(dim)

# Cinemática Inversa: X = x - w
f_inv = I - fd.grad(w)
det_f = fd.det(f_inv)
F = fd.inv(f_inv) 
J = fd.det(F)

# Constitutivo
C = F.T * F
invC = fd.inv(C)
E = fd.variable(0.5 * (C - I))

J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))
Psi_el = c_fung * (fd.exp(a_fung * J1**2 + b_fung * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

# Cargas Externas
ratio = Gamma_field / Gamma_inf
gamma_lang = gamma_0 - m1 * ratio
gamma_insol = gamma_inf - m2 * (ratio - 1.0)
gamma_base = fd.conditional(fd.lt(Gamma_field, Gamma_inf), gamma_lang, gamma_insol)
gamma_base = fd.max_value(gamma_base, gamma_min)

gamma_eff = alpha_load * gamma_base
Phi = fd.max_value(J - 1.0 + Phi_R, 1e-4)
P_gamma = (2.0 * gamma_eff / R_RI) * ((Phi_R / Phi)**(1.0/3.0))
P_alv_eff = alpha_load * P_alv_final

S_ext = (P_gamma - P_alv_eff) * J * invC
S_total = S_el + S_ext

# Residuo
P_stress = F * S_total
grad_v_ref = F.T * fd.grad(v_test)
Residual = fd.inner(P_stress, grad_v_ref) * det_f * fd.dx

bcs = [
    fd.DirichletBC(V, fd.Constant((0.0, 0.0)), 1),
    fd.DirichletBC(V.sub(1), 0.0, 3)
]

# ============================================================
# 6. RESOLUCIÓN
# ============================================================

problem = fd.NonlinearVariationalProblem(Residual, w, bcs=bcs)
solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt", 
    "snes_max_it": 50,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
    "snes_monitor": None
}
solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

w.assign(fd.Constant((0.0, 0.0)))
print("\nIniciando cálculo inverso...")
n_steps = 10
success = True

for step in range(n_steps + 1):
    alpha_val = step / n_steps
    alpha_load.assign(alpha_val)
    print(f"--- Carga {step}/{n_steps}: {alpha_val*100:.0f}% ---")
    try:
        solver.solve()
        w_max = np.max(np.abs(w.dat.data_ro))
        print(f"    Convergió. w_max: {w_max:.4f} mm")
    except fd.ConvergenceError:
        print(f"!!! Falló convergencia !!!")
        success = False
        break

# ============================================================
# 7. GENERACIÓN DE RESULTADOS PARA PARAVIEW
# ============================================================

output_dir = "phase2_results"
os.makedirs(output_dir, exist_ok=True)

if success:
    print("\n" + "="*60)
    print("GUARDANDO RESULTADO FINAL")
    print("="*60)
    
    # --- 1. Crear el vector de RECUPERACIÓN ---
    # Para "Warp by Vector" en Paraview, necesitamos un vector tal que:
    # Pos_Final = Pos_Inicial + Vector
    # Queremos: Pos_Final = StressFree
    # Sabemos: StressFree = Deformada - w
    # Por tanto: StressFree = Deformada + (-w)
    # EL VECTOR ES -w
    
    u_recovery = fd.Function(V, name="Recovery_Vector")
    u_recovery.assign(-1.0 * w)
    
    # --- 2. Guardar en formato PVD sobre la malla deformada ---
    outfile = VTKFile(os.path.join(output_dir, "inverse_visual_ready.pvd"))
    outfile.write(u_recovery, w)
    
    print(f"Archivo generado: {output_dir}/inverse_visual_ready.pvd")
    print("\n*** INSTRUCCIONES PARA PARAVIEW ***")
    print("1. Abre 'inverse_visual_ready.pvd'.")
    print("2. Verás la malla DEFORMADA (expandida).")
    print("3. Aplica el filtro 'Warp By Vector'.")
    print("4. Selecciona 'Vectors': 'Recovery_Vector'.")
    print("5. Deja 'Scale Factor': 1.0.")
    print("6. Dale a 'Apply' -> La malla volverá a ser un cuadrado.")
    
    # --- 3. Verificación Numérica (Solución al error de broadcasting) ---
    # Interpolamos w a CG1 (lineal) para poder restarlo de las coordenadas (CG1)
    V_coords = mesh_obs.coordinates.function_space()
    w_cg1 = fd.Function(V_coords)
    w_cg1.interpolate(w)
    
    # Calculamos coordenadas recuperadas
    coords_rec = mesh_obs.coordinates.dat.data_ro - w_cg1.dat.data_ro
    
    # Construimos malla temporal para medir área
    mesh_rec_check = fd.Mesh(mesh_topology.coordinates)
    mesh_rec_check.coordinates.dat.data[:] = coords_rec
    
    vol_obs = fd.assemble(fd.Constant(1.0)*fd.dx(domain=mesh_obs))
    vol_rec = fd.assemble(fd.Constant(1.0)*fd.dx(domain=mesh_rec_check))
    
    print(f"\nVerificación:")
    print(f"  Área Inicial (Deformada): {vol_obs:.2f} mm²")
    print(f"  Área Recuperada (Calculada): {vol_rec:.2f} mm²")
    
else:
    print("\nEl cálculo falló.")

FASE 2: INVERSE MOTION SOBRE MALLA DEFORMADA
Cargando datos desde phase1_results/forward_data.npz ...
Entradas cargadas:
  - Geometría: 441 nodos
  - Carga externa: P_alv = 1000.0 Pa
Malla deformada reconstruida exitosamente.

Iniciando cálculo inverso...
--- Carga 0/10: 0% ---
  0 SNES Function norm 0.000000000000e+00
    Convergió. w_max: 0.0000 mm
--- Carga 1/10: 10% ---
  0 SNES Function norm 3.075869615248e-01
  1 SNES Function norm 1.142434338038e-01
  2 SNES Function norm 1.063075962059e-01
  3 SNES Function norm 9.108184223706e-02
  4 SNES Function norm 4.620321347113e-02
  5 SNES Function norm 3.312500506346e-02
  6 SNES Function norm 4.928043268192e-03
  7 SNES Function norm 1.638995647707e-04
  8 SNES Function norm 6.934840453414e-08
  9 SNES Function norm 1.262946092093e-14
    Convergió. w_max: 0.4914 mm
--- Carga 2/10: 20% ---
  0 SNES Function norm 3.082200787596e-01
  1 SNES Function norm 6.959737912085e-02
  2 SNES Function norm 4.051320957673e-02
  3 SNES Function nor

In [ ]:
"""
HERRAMIENTA DE DIAGNÓSTICO Y CONVERSIÓN DE MALLA (H5 -> PVD)
============================================================
Objetivo:
  1. Leer manualmente los arrays de coordenadas y topología del H5.
  2. Reconstruir la malla en memoria (Bypasseando lectores automáticos).
  3. Guardar en formato .pvd para visualizar en ParaView.

Este método es "a prueba de balas" contra errores de formato de archivo.
"""

import os
import numpy as np
import firedrake as fd
from firedrake.output import VTKFile
from petsc4py import PETSc
import h5py

# ============================================================
# 1. SELECCIÓN DE ARCHIVO
# ============================================================

MESH_FILE = "mesh.h5"
if not os.path.exists(MESH_FILE):
    MESH_FILE = "coarse.h5"

print("="*60)
print(f"CONVERSIÓN MANUAL DE MALLA: {MESH_FILE}")
print("="*60)

# ============================================================
# 2. LECTURA MANUAL DE DATOS (H5PY)
# ============================================================

print("\n--- Leyendo datos crudos con h5py ---")

coords_data = None
cells_data = None

try:
    with h5py.File(MESH_FILE, 'r') as f:
        # Inspección rápida
        if 'mesh' not in f:
            raise RuntimeError("El archivo no contiene el grupo '/mesh'.")
        
        # Leer Coordenadas
        if 'coordinates' in f['mesh']:
            coords_data = f['mesh']['coordinates'][:]
            print(f"  > Coordenadas leídas. Shape: {coords_data.shape}")
        else:
            raise RuntimeError("No se encontró '/mesh/coordinates'.")
            
        # Leer Topología (Celdas/Conectividad)
        if 'topology' in f['mesh']:
            cells_data = f['mesh']['topology'][:]
            print(f"  > Topología leída. Shape: {cells_data.shape}")
        else:
            raise RuntimeError("No se encontró '/mesh/topology'.")

except Exception as e:
    print(f"!!! Error leyendo el archivo H5: {e}")
    raise

# ============================================================
# 3. RECONSTRUCCIÓN DE LA MALLA (PETSc DMPlex)
# ============================================================

print("\n--- Reconstruyendo objeto Mesh ---")

# Determinar dimensión topológica
# Si las celdas tienen 3 nodos -> Triángulos (2D)
# Si las celdas tienen 4 nodos -> Tetraedros (3D)
num_nodes_per_cell = cells_data.shape[1]

if num_nodes_per_cell == 3:
    dim = 2
    print("  Detectado: Malla 2D (Triángulos)")
elif num_nodes_per_cell == 4:
    dim = 3
    print("  Detectado: Malla 3D (Tetraedros)")
else:
    raise ValueError(f"Tipo de celda no soportado (nodos por celda: {num_nodes_per_cell})")

# Crear DMPlex desde la lista de celdas
# createFromCellList(dimensión_topologica, celdas, coordenadas)
try:
    plex = PETSc.DMPlex().createFromCellList(dim, cells_data, coords_data, comm=PETSc.COMM_WORLD)
    
    # Crear malla de Firedrake
    mesh = fd.Mesh(plex)
    print("  > Malla creada exitosamente en Firedrake.")
    
except Exception as e:
    print(f"!!! Error construyendo DMPlex: {e}")
    raise

# ============================================================
# 4. REPORTE Y GUARDADO
# ============================================================

# Estadísticas
n_cells = mesh.num_cells()
n_verts = mesh.num_vertices()
dim_geo = mesh.geometric_dimension()

print("\n--- Estadísticas Finales ---")
print(f"  Dimensión Geométrica: {dim_geo}D")
print(f"  Elementos: {n_cells}")
print(f"  Nodos:     {n_verts}")

coords = mesh.coordinates.dat.data_ro
print(f"  Rango X: [{coords[:,0].min():.2f}, {coords[:,0].max():.2f}]")
print(f"  Rango Y: [{coords[:,1].min():.2f}, {coords[:,1].max():.2f}]")
if dim_geo == 3:
    print(f"  Rango Z: [{coords[:,2].min():.2f}, {coords[:,2].max():.2f}]")

# Guardar PVD
output_dir = "phase3_results"
os.makedirs(output_dir, exist_ok=True)

pvd_path = os.path.join(output_dir, "converted_mesh.pvd")
print(f"\n--- Guardando visualización ---")
print(f"  Archivo: {pvd_path}")

try:
    outfile = VTKFile(pvd_path)
    outfile.write(mesh.coordinates)
    print("\n¡ÉXITO! Conversión completada.")
    print("Ahora abre 'phase3_results/converted_mesh.pvd' en ParaView.")
except Exception as e:
    print(f"Error escribiendo PVD: {e}")

FASE 2: INVERSE MOTION EN PULMÓN 3D
Malla observada (CT):
  Elementos: 28458
  Nodos: 5796
  X: [-65.27, 65.27]
  Y: [-85.97, 85.97]
  Z: [-106.72, 106.72]
  Volumen: 1573075.40 mm³ = 1.5731 L

Parámetros:
  c = 2.5 kPa
  P_pleural = -0.5 kPa (estimada)
  DoFs: 17388

RESOLVIENDO PROBLEMA INVERSO

Paso 1/10: carga = 10%


LookupError: BC construction got invalid markers {<function bottom_boundary at 0x707b3a0d6840>}. Valid markers are 'set()'

In [20]:
"""
FASE 2 (3D): Inverse Motion sobre Pulmón Real
==============================================
"""

import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import firedrake as fd
from firedrake.output import VTKFile
from petsc4py import PETSc
import h5py

fd.parameters["form_compiler"]["quadrature_degree"] = 4

# ============================================================
# 1. CARGAR MALLA
# ============================================================

MESH_FILE = "mesh.h5"

print("="*60)
print("FASE 2: INVERSE MOTION EN PULMÓN 3D")
print("="*60)

with h5py.File(MESH_FILE, 'r') as f:
    coords_data = f['mesh']['coordinates'][:].astype(np.float64)
    cells_data = f['mesh']['topology'][:].astype(np.int32)

plex = PETSc.DMPlex().createFromCellList(3, cells_data, coords_data, comm=PETSc.COMM_WORLD)
mesh_obs = fd.Mesh(plex)

coords_obs = mesh_obs.coordinates.dat.data_ro.copy()

print(f"Malla: {mesh_obs.num_cells()} elementos, {mesh_obs.num_vertices()} nodos")
print(f"  X: [{coords_obs[:,0].min():.2f}, {coords_obs[:,0].max():.2f}]")
print(f"  Y: [{coords_obs[:,1].min():.2f}, {coords_obs[:,1].max():.2f}]")
print(f"  Z: [{coords_obs[:,2].min():.2f}, {coords_obs[:,2].max():.2f}]")

V_obs = fd.assemble(fd.Constant(1.0) * fd.dx(mesh_obs))
print(f"  Volumen: {V_obs/1e6:.4f} L")

# ============================================================
# 2. PARÁMETROS
# ============================================================

Phi_R = 0.74
R_RI = 0.11

c_fung = 2.5
a_fung = 0.433
b_fung = -0.61

gamma_0 = 70.0e-6
gamma_inf = 22.2e-6
gamma_min = 0.0
Gamma_inf = 3.0e-9
m1 = 47.8e-6
m2 = 140.0e-6
Gamma_max = Gamma_inf * (1.0 + (gamma_inf - gamma_min) / m2)

P_pleural = -0.5  # kPa
P_alv_obs = -P_pleural

# ============================================================
# 3. ESPACIOS Y VARIABLES
# ============================================================

V = fd.VectorFunctionSpace(mesh_obs, "CG", 1)
Q = fd.FunctionSpace(mesh_obs, "DG", 0)

w = fd.Function(V, name="Inverse_Displacement")
v_test = fd.TestFunction(V)

Gamma = fd.Function(Q, name="Surfactant")
Gamma.assign(fd.Constant(0.5 * (Gamma_inf + Gamma_max)))

alpha_load = fd.Constant(0.0)

print(f"  DoFs: {V.dim()}")

# ============================================================
# 4. CONDICIONES DE BORDE (SIN MARCADORES)
# ============================================================

z_min = coords_obs[:, 2].min()
z_max = coords_obs[:, 2].max()
z_range = z_max - z_min

# Identificar nodos en la base (z pequeño)
tol = 0.02 * z_range
fixed_node_ids = np.where(coords_obs[:, 2] < z_min + tol)[0]
print(f"  Nodos fijos (base): {len(fixed_node_ids)}")

# Crear BC usando el método de nodos específicos
# En Firedrake, podemos usar una expresión condicional en el residuo
# o penalización débil

# Método: Penalización fuerte via modificación del residuo
# Alternativa más robusta: fijar unos pocos nodos para evitar modos rígidos

# Encontrar nodo más cercano al centroide de la base
base_centroid = coords_obs[fixed_node_ids].mean(axis=0)
distances = np.linalg.norm(coords_obs - base_centroid, axis=1)
anchor_node = np.argmin(distances)
print(f"  Nodo ancla: {anchor_node}, coords: {coords_obs[anchor_node]}")

# ============================================================
# 5. MODELO CONSTITUTIVO
# ============================================================

dim = 3
I = fd.Identity(dim)

f_inv = I - fd.grad(w)
det_f = fd.det(f_inv)
F = fd.inv(f_inv)
J = fd.det(F)

C = F.T * F
invC = fd.inv(C)
E = fd.variable(0.5 * (C - I))

J1 = fd.tr(E)
J2 = 0.5 * (fd.tr(E)**2 - fd.tr(E * E))

Psi_el = c_fung * (fd.exp(a_fung * J1**2 + b_fung * J2) - 1.0)
S_el = fd.diff(Psi_el, E)

ratio = Gamma / Gamma_inf
gamma_lang = gamma_0 - m1 * ratio
gamma_insol = gamma_inf - m2 * (ratio - 1.0)
gamma_surf = fd.conditional(fd.lt(Gamma, Gamma_inf), gamma_lang, gamma_insol)
gamma_surf = fd.max_value(gamma_surf, gamma_min)

gamma_eff = alpha_load * gamma_surf
Phi = fd.max_value(J - 1.0 + Phi_R, 1e-4)
P_gamma = (2.0 * gamma_eff / R_RI) * ((Phi_R / Phi)**(1.0/3.0))
P_alv_eff = alpha_load * P_alv_obs

S_ext = (P_gamma - P_alv_eff) * J * invC
S_total = S_el + S_ext

# ============================================================
# 6. FORMULACIÓN CON PENALIZACIÓN
# ============================================================

P_stress = F * S_total
grad_v_ref = F.T * fd.grad(v_test)
det_f_safe = fd.max_value(det_f, 1e-6)

# Residuo principal
Residual = fd.inner(P_stress, grad_v_ref) * det_f_safe * fd.dx

# Penalización para nodos fijos (método de penalización)
# Creamos una función indicadora para la región fija
x = fd.SpatialCoordinate(mesh_obs)
indicator = fd.conditional(fd.lt(x[2], z_min + tol), 1.0, 0.0)

beta_penalty = fd.Constant(1e6)  # Factor de penalización grande
Residual += beta_penalty * indicator * fd.inner(w, v_test) * fd.dx

# Sin BCs explícitas (usamos penalización)
bcs = []

problem = fd.NonlinearVariationalProblem(Residual, w, bcs=bcs)

solver_params = {
    "snes_type": "newtonls",
    "snes_linesearch_type": "bt",
    "snes_max_it": 50,
    "snes_atol": 1e-6,
    "snes_rtol": 1e-5,
    "snes_monitor": None,
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}

solver = fd.NonlinearVariationalSolver(problem, solver_parameters=solver_params)

# ============================================================
# 7. RESOLVER
# ============================================================

print("\n" + "="*60)
print("RESOLVIENDO PROBLEMA INVERSO")
print("="*60)

w.assign(fd.Constant((0.0, 0.0, 0.0)))

n_steps = 10
success = True

for step in range(1, n_steps + 1):
    alpha_val = step / n_steps
    alpha_load.assign(alpha_val)
    
    print(f"\nPaso {step}/{n_steps}: carga = {alpha_val*100:.0f}%")
    
    try:
        solver.solve()
        w_norm = fd.norm(w)
        w_max = np.max(np.abs(w.dat.data_ro))
        print(f"  ✓ ||w|| = {w_norm:.4f}, |w|_max = {w_max:.4f} mm")
    except fd.ConvergenceError:
        print(f"  ✗ No convergió")
        success = False
        break

# ============================================================
# 8. RESULTADOS
# ============================================================

output_dir = "phase2_results_3d"
os.makedirs(output_dir, exist_ok=True)

if success:
    print("\n" + "="*60)
    print("GUARDANDO RESULTADOS")
    print("="*60)
    
    V_coords = mesh_obs.coordinates.function_space()
    w_interp = fd.interpolate(w, V_coords)
    coords_stressfree = coords_obs - w_interp.dat.data_ro
    
    u_recovery = fd.Function(V, name="Recovery_Vector")
    u_recovery.assign(-1.0 * w)
    
    outfile = VTKFile(os.path.join(output_dir, "lung_inverse_result.pvd"))
    w.rename("Inverse_Displacement")
    outfile.write(u_recovery, w)
    
    mesh_stressfree = fd.Mesh(mesh_obs.coordinates.copy(deepcopy=True))
    mesh_stressfree.coordinates.dat.data[:] = coords_stressfree
    
    sf_file = VTKFile(os.path.join(output_dir, "lung_stressfree.pvd"))
    V_sf = fd.VectorFunctionSpace(mesh_stressfree, "CG", 1)
    zero_sf = fd.Function(V_sf, name="zero")
    sf_file.write(zero_sf)
    
    np.savez(os.path.join(output_dir, "inverse_data_3d.npz"),
             w_data=w.dat.data_ro[:],
             coords_obs=coords_obs,
             coords_stressfree=coords_stressfree)
    
    V_sf_vol = fd.assemble(fd.Constant(1.0) * fd.dx(mesh_stressfree))
    displacement_mag = np.linalg.norm(w.dat.data_ro, axis=1)
    
    print(f"\nArchivos en: {output_dir}/")
    print(f"\nEstadísticas:")
    print(f"  Vol observado: {V_obs/1e6:.4f} L")
    print(f"  Vol stress-free: {V_sf_vol/1e6:.4f} L")
    print(f"  Ratio: {V_obs/V_sf_vol:.4f}")
    print(f"  |w| máx: {displacement_mag.max():.4f} mm")
    print(f"  |w| prom: {displacement_mag.mean():.4f} mm")

else:
    print("\nFalló. Reducir P_pleural o aumentar n_steps.")

print("\nFASE 2 COMPLETADA")

FASE 2: INVERSE MOTION EN PULMÓN 3D
Malla: 28458 elementos, 5796 nodos
  X: [-65.27, 65.27]
  Y: [-85.97, 85.97]
  Z: [-106.72, 106.72]
  Volumen: 1.5731 L
  DoFs: 17388
  Nodos fijos (base): 10
  Nodo ancla: 0, coords: [ -25.04871491  -67.79529567 -106.51972466]

RESOLVIENDO PROBLEMA INVERSO

Paso 1/10: carga = 10%
  0 SNES Function norm 1.001353440490e+02
  1 SNES Function norm 1.186696828255e+00
  2 SNES Function norm 4.390696177951e-02
  3 SNES Function norm 9.232925599075e-03
  4 SNES Function norm 3.100140979442e-06
  ✓ ||w|| = 2500.4775, |w|_max = 2.5711 mm

Paso 2/10: carga = 20%
  0 SNES Function norm 1.001364986013e+02
  1 SNES Function norm 1.280212663385e+00
  2 SNES Function norm 2.954510533222e-02
  3 SNES Function norm 9.765012449403e-03
  4 SNES Function norm 2.108227600699e-06
  ✓ ||w|| = 4945.0956, |w|_max = 5.1142 mm

Paso 3/10: carga = 30%
  0 SNES Function norm 1.001384110205e+02
  1 SNES Function norm 1.453639495624e+00
  2 SNES Function norm 4.815544085673e-02
  

/home/ebasu/venv-firedrake/lib/python3.12/site-packages/firedrake/interpolation.py:391: FutureWarning: The use of `interpolate` to perform the numerical interpolation is deprecated.
This feature will be removed very shortly.

Instead, import `interpolate` from the `firedrake.__future__` module to update
the interpolation's behaviour to return the symbolic `ufl.Interpolate` object associated
with this interpolation.

You can then assemble the resulting object to get the interpolated quantity
of interest. For example,

```
from firedrake.__future__ import interpolate
...

assemble(interpolate(expr, V))
```

Alternatively, you can also perform other symbolic operations on the interpolation operator, such as taking
the derivative, and then assemble the resulting form.

  warnings.warn("""The use of `interpolate` to perform the numerical interpolation is deprecated.
